In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install glob2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from glob import glob
import os
import pandas as pd
path = "/content/drive/MyDrive/Iqram Sir/Fitabase data pulls - seperate"

In [4]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm.notebook import tqdm

def get_content(patient_folder_adrs):

  post_adrs = sorted(glob(patient_folder_adrs+"/*"))[0]
  

  pre_adrs = sorted(glob(patient_folder_adrs+"/*"))[1]

  for i in glob(post_adrs+"/*"):
    content = i.split("_")
    if "heartrate" in content and "seconds" in content:
      post_sec = i
    if "minuteMETsNarrow" in content:
      post_met = i


  for i in glob(pre_adrs+"/*"):
    content = i.split("_")
    if "heartrate" in content and "seconds" in content:
      pre_sec = i
    if "minuteMETsNarrow" in content:
      pre_met = i
 
  return post_sec,post_met, pre_sec, pre_met





def signle_sample(df_1min,df_met):
  list_1 = list(df_1min["Time"])
  list_3 = list(df_met["ActivityMinute"])


  hr = []
  time = []
  mets = []

  if len(list_1)>len(list_3):
    for i in tqdm(list_1):
      if  i in list_3:
        hr.append(list(df_1min.query("Time == '"+i+"'")['Value'])[0])
        mets.append(list(df_met.query("ActivityMinute == '"+i+"'")['METs'])[0])
        time.append(i)

  else:
    for i in tqdm(list_3):
      if i in list_1:
        hr.append(list(df_1min.query("Time == '"+i+"'")['Value'])[0])
        mets.append(list(df_met.query("ActivityMinute == '"+i+"'")['METs'])[0])
        time.append(i)

  d={
    "Time" : time,
     "HR(BPM)" : hr,
     "METs" : mets
  }

  dataFrame = pd.DataFrame.from_dict(d)
  return dataFrame


def test_signle_sample(data,df_1min,df_met):
  total_row = data.shape[0]
  print("Total DataRow: ",total_row)

  err_row=[]

  for i in tqdm(range(total_row)):
    print(data["Time"][i], data['HR(BPM)'][i],data['METs'][i])

    hr = list(df_1min.query("Time == '"+data["Time"][i]+"'")['Value'])[0]
    mets = list(df_met.query("ActivityMinute == '"+data["Time"][i]+"'")['METs'])[0]
    print(hr,mets)
    print("------------------------------------------------------------")

    if hr != data['HR(BPM)'][i] or mets != data['METs'][i]:
      err_row.append((data["Time"][i],i))
  return err_row




def range_list(k):
  list_of = []
  main_list = []

  for i in range(len(k)-1):
 
    if k[i+1]-k[i] == 1:
      list_of.append(k[i])
    else:
      list_of.append(k[i])
      main_list.append(list_of)
      list_of=[]
    if i == len(k)-2:
      if k[i+1]-k[i] == 1:
        list_of.append(k[i+1])
        main_list.append(list_of)
      else:
        main_list.append([k[i+1]])
  
  return main_list






def mets_minute_refine(data,minutes,upper_lim,lower_lim):

  mets_none = []
  mets_more = []
  for i in range(data.shape[0]):

    if lower_lim< data["METs"][i] <= upper_lim:
    #if data["METs"][i] > 10:
      mets_more.append(i)
    else:
      mets_none.append(i)

  main_l = range_list(mets_more)

  removable_list = []
  for i in range(len(main_l)-1):
    if len(main_l[i]) < minutes+1:
      for j in main_l[i]:
        removable_list.append(j)

  df1=data.drop(data.index[mets_none+removable_list]).reset_index()
  return df1





from datetime import datetime

def formate24(t):
  time = t.split(" ")
  m2 = time[1]+" "+time[2]
  in_time = datetime.strptime(m2, "%I:%M:%S %p")
  out_time = datetime.strftime(in_time, "%H:%M:%S")
  return out_time


def dif_min(start_time, end_time):
  t1 = datetime.strptime(start_time, "%H:%M:%S")
  t2 = datetime.strptime(end_time, "%H:%M:%S")

  delta = t2 - t1

  return delta.total_seconds()/60







def time_limit_check(test_data,min_limit):
  time_diff_list = []
  let = range_list(list(test_data['index']))
  for i in tqdm(let):

    start_time = formate24(list(test_data.query("index == "+str(i[0])+"")['Time'])[0])
    end_time = formate24(list(test_data.query("index == "+str(i[-1])+"")['Time'])[0])
    diff = dif_min(start_time,end_time)
    if diff < min_limit:
      time_diff_list.append(i)
  
  return time_diff_list


def single_segment_refine(single_list,min_limit):
  temp = []
  main = []
  if len(single_list)>1:
    for i in range(len(single_list)-1):
      k = formate24(list(test_data.query("index == "+str(single_list[i+1])+"")['Time'])[0])
      l = formate24(list(test_data.query("index == "+str(single_list[i])+"")['Time'])[0])
      diff = dif_min(l,k)
      #print(k,l,diff)
      if diff == 1 or diff == -1439.0:
        temp.append(single_list[i])
        flage = 1
      else:
        temp.append(single_list[i])
        if len(temp) >= min_limit+1:
          main.append(temp)
        temp=[]
        flage = 0

    if flage == 1:
      if len(temp) >= min_limit+1:
        k = formate24(list(test_data.query("index == "+str(single_list[i+1])+"")['Time'])[0])
        l = formate24(list(test_data.query("index == "+str(single_list[i])+"")['Time'])[0])
        diff = dif_min(l,k)
        if diff == 1 or diff == -1439.0:
          temp.append(single_list[i+1])
          main.append(temp)
        else:
          main.append(temp)


  return main


def final_refination(data,min_lim):
  min_lim = 5
  print("Time Limit Checking:")
  time_diff_list = time_limit_check(data,min_lim)
  print()

  removeable_list=[]

  print("Unwanted index list making:")
  for i in tqdm(time_diff_list):
  # print(i)
    lis = single_segment_refine(i,min_lim)
    combined = []

    for j in lis:
      combined=combined+j


    for k in i:
      if k not in combined:
        removeable_list.append(k)
  

  print()
  print("Refining Dataset:")

  for i in tqdm(removeable_list):
    data.drop(data.loc[data['index']==i].index, inplace=True)

  data.reset_index()

  return data


In [5]:
subject = '001-JV'



for i in glob(path+"/*"):
  if subject in i.split("/"):
    Post_sec,Post_met, Pre_sec,Pre_met = get_content(i)

df_post_sec = pd.read_csv(Post_sec)
df_post_met = pd.read_csv(Post_met)

df_pre_sec = pd.read_csv(Pre_sec)
df_pre_met = pd.read_csv(Pre_met)


In [6]:
df_pre_sec.head()

,Time,Value
0,12/7/2021 3:52:33 PM,70
1,12/7/2021 3:52:48 PM,70
2,12/7/2021 3:52:58 PM,67
3,12/7/2021 3:53:03 PM,65
4,12/7/2021 3:53:18 PM,65


In [7]:
df_pre_met.head()

,ActivityMinute,METs
0,12/3/2021 12:00:00 AM,10
1,12/3/2021 12:01:00 AM,10
2,12/3/2021 12:02:00 AM,10
3,12/3/2021 12:03:00 AM,10
4,12/3/2021 12:04:00 AM,10


In [ ]:
t1 = df_pre_sec['Time'][0]

In [ ]:
t2 = df_pre_met['ActivityMinute'][5000]

In [ ]:
print(t1,t2)

12/7/2021 3:52:33 PM 12/6/2021 11:20:00 AM


In [ ]:
from datetime import datetime

def compare_dates(date1, date2):
    # convert string to date
    dt_obj1 = datetime.strptime(date1, "%Y-%m-%d %H:%M:%S")
    dt_obj2 = datetime.strptime(date2, "%Y-%m-%d %H:%M:%S")
    print('Date1:', dt_obj1)
    print('Date2:', dt_obj2)

    if dt_obj1 == dt_obj2:
        print('Both dates are equal')
    elif dt_obj1 > dt_obj2:
        print('Date1 is greater than date2')
    else:
        print('Date1 is lower than date2')

# datetime in yyyy-mm-dd hh:mm:ss format
dt_str1 = '2022-10-29 8:32:49'
dt_str2 = '2022-5-7 4:14:58'
compare_dates(dt1_str1, dt2_str2)

NameError: ignored

In [ ]:
time = t.split(" ")
m2 = time[1]+" "+time[2]
in_time = datetime.strptime(m2, "%I:%M:%S %p")
out_time = datetime.strftime(in_time, "%H:%M")


In [ ]:
print(in_time)
print(out_time)

1900-01-01 15:52:33
15:52


In [8]:
def remove_sec(l):
  k = datetime.strptime(l, "%m/%d/%Y %H:%M:%S %p")
  out_time = datetime.strftime(k, "%-m/%-d/%Y %-H:%M %p")
  return out_time


def add_sec(l):
  k = datetime.strptime(l, "%m/%d/%Y %H:%M %p")
  out_time = datetime.strftime(k, "%-m/%-d/%Y %-H:%M:%S %p")
  return out_time

In [ ]:
li_met = list(df_pre_met['ActivityMinute'])
li_met_s = []
li_sec = list(df_pre_sec['Time'])

for i in li_met:
  li_met_s.append(remove_sec(i))


In [ ]:
activity_time=[]
hr=[]
mets=[]


for i in tqdm(li_sec):
  if remove_sec(i) in li_met_s:
    activity_time.append(i)
    hr.append(list(df_pre_sec.query("Time == '"+i+"'")['Value'])[0])
    mets.append(list(df_pre_met.query("ActivityMinute == '"+add_sec(remove_sec(i))+"'")['METs'])[0])
 

d={
  "ActivityMinute" : activity_time,
  "HR(BPM)" : hr,
  "METs" : mets
}

dataFrame = pd.DataFrame.from_dict(d)


In [ ]:
add_sec(remove_sec(i))

'12/10/2021 1:00:00 AM'

In [ ]:
li_met_s.index(remove_sec(i))

10140

In [ ]:
li_met[li_met_s.index(remove_sec(i))]

'12/10/2021 1:00:00 AM'

In [ ]:
list(df_pre_met.query("ActivityMinute == '"+add_sec(remove_sec(i))+"'")['METs'])

[]

In [ ]:
print(remove_sec(li_met[0]))

12/03/2021 12:00 PM


In [ ]:
 print(add_sec(remove_sec(li_sec[0])))

12/07/2021 03:52:00 AM


In [11]:
li_met = list(df_pre_met['ActivityMinute'])
li_sec = list(df_pre_sec['Time'])

len_met = len(li_met)
len_sec = len(li_sec)

metO = datetime.strptime(li_met[0], "%m/%d/%Y %H:%M:%S %p")
secO = datetime.strptime(li_sec[0], "%m/%d/%Y %H:%M:%S %p")

lim = 0

if metO > secO:
  print("y")

else:
  print(li_met[0],li_sec[0])




12/3/2021 12:00:00 AM 12/7/2021 3:52:33 PM


In [10]:
print(li_met[0],li_sec[0])

12/3/2021 12:00:00 AM 12/7/2021 3:52:33 PM


In [ ]:
dt_obj1

datetime.datetime(2021, 12, 3, 12, 0)

In [ ]:
if 